In [37]:
import joblib
from prophet import Prophet
from datetime import datetime, timedelta
from yahoo_fin import stock_info as si
import pandas as pd
import matplotlib.pyplot as plt
from prophet.diagnostics import cross_validation, performance_metrics

# Display plots inline
%matplotlib inline
    

In [38]:
# List of stocks
stocks = ["AAPL", "ABBV", "ADBE", "AMZN", "AVGO", "BRK-B", "CRM", "COST", "CVX", "HD", 
          "JNJ", "JPM", "LLY", "MA", "META", "MRK", "MSFT", "NVDA", "PG", "TSLA", "UNH", "V", "XOM"]


In [39]:
def fetch_data(stock):
    end_date = datetime(2024, 2, 9)
    start_date = end_date - timedelta(days=2*365)
    data = si.get_data(stock, start_date=start_date, end_date=end_date)
    data.reset_index(inplace=True)
    data.rename(columns={'index': 'date'}, inplace=True)
    return data

In [40]:
def calculate_technical_indicators(df):
    # Calculate Simple Moving Average (SMA)
    df['SMA_10'] = df['close'].rolling(window=10).mean()

    # Calculate Relative Strength Index (RSI)
    delta = df['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # Calculate Bollinger Bands
    df['Upper_Band'] = df['close'].rolling(window=20).mean() + 2 * df['close'].rolling(window=20).std()
    df['Lower_Band'] = df['close'].rolling(window=20).mean() - 2 * df['close'].rolling(window=20).std()

    # Calculate lagged closing prices
    df['Close_1'] = df['close'].shift(1)
    df['Close_2'] = df['close'].shift(2)

    df.dropna(inplace=True)
    return df

In [41]:
def prepare_data_for_prophet(df):
    df_prophet = df[['date', 'close', 'SMA_10', 'RSI_14', 'Upper_Band', 'Lower_Band', 'Close_1', 'Close_2']]
    df_prophet.rename(columns={'date': 'ds', 'close': 'y'}, inplace=True)
    return df_prophet

In [42]:
def evaluate_model(model, data, horizon='90 days', period='30 days', initial='365 days'):
    df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon, parallel="processes")
    df_p = performance_metrics(df_cv)
    return df_p

In [45]:
def find_optimal_retraining_period(stock, model_path, data_prophet):
    retraining_periods = ['30 days', '60 days', '90 days', '120 days']
    best_rmse = float('inf')
    best_period = None
    performance_results = {}

    try:
        model = joblib.load(model_path)
    except FileNotFoundError:
        print(f"Model for {stock} not found. Skipping.")
        return None, None

    for period in retraining_periods:
        df_p = evaluate_model(model, data_prophet, period=period)
        avg_rmse = df_p['rmse'].mean()
        performance_results[period] = df_p
        if avg_rmse < best_rmse:
            best_rmse = avg_rmse
            best_period = period

    return best_period, performance_results

In [46]:
# Dictionary to store optimal retraining periods for each stock
optimal_retraining_periods = {}

# Loop through each stock
for stock in stocks:
    print(f"Processing {stock}...")
    
    # Fetch historical data
    data = fetch_data(stock)
    
    # Calculate technical indicators
    data = calculate_technical_indicators(data)
    
    # Prepare the data for Prophet
    data_prophet = prepare_data_for_prophet(data)
    
    # Load the corresponding Prophet model (assuming you have a separate model for each stock)
    model_path = f'Models/{stock}_prophet_model.pkl'
    
    # Find the optimal retraining period
    best_period, performance_results = find_optimal_retraining_period(stock, model_path, data_prophet)
    
    if best_period:
        optimal_retraining_periods[stock] = best_period
        print(f"Optimal retraining period for {stock}: {best_period}")

        # # Plotting the performance metrics for each period
        # for period, df_p in performance_results.items():
        #     plt.figure(figsize=(12, 6))
        #     plt.plot(df_p['horizon'], df_p['rmse'], label='RMSE')
        #     plt.plot(df_p['horizon'], df_p['mape'], label='MAPE')
        #     plt.plot(df_p['horizon'], df_p['mae'], label='MAE')
        #     plt.xlabel('Horizon (days)')
        #     plt.ylabel('Error')
        #     plt.legend()
        #     plt.title(f'Model Performance for {stock} with {period} Retraining Period')
        #     plt.savefig(f'performance_{stock}_{period}.png')
        #     plt.close()  # Close the plot to avoid display issues
        
        # Display the performance metrics for the best retraining period
        best_metrics = performance_results[best_period]
        print(f"Performance metrics for {stock} with optimal retraining period ({best_period}):")
        display(best_metrics)
    else:
        print(f"Skipping {stock} due to missing model.")

Processing AAPL...


C:\Users\kahbe\AppData\Local\Temp\ipykernel_1320\2948041207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prophet.rename(columns={'date': 'ds', 'close': 'y'}, inplace=True)


Optimal retraining period for AAPL: 60 days
Performance metrics for AAPL with optimal retraining period (60 days):


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,8 days,3.118887,1.766037,1.523264,0.008716,0.007303,0.008717,0.916667
1,9 days,4.335611,2.082213,1.738129,0.009995,0.007303,0.009963,0.833333
2,10 days,4.009124,2.002280,1.624719,0.009328,0.007303,0.009290,0.833333
3,11 days,3.903966,1.975846,1.526147,0.008711,0.007303,0.008670,0.833333
4,12 days,4.845460,2.201241,1.718961,0.009810,0.008273,0.009786,0.750000
...,...,...,...,...,...,...,...,...
77,86 days,1.244532,1.115586,0.894358,0.004933,0.004082,0.004947,1.000000
78,87 days,1.140521,1.067952,0.816266,0.004499,0.002947,0.004512,1.000000
79,88 days,2.120625,1.456237,1.081260,0.005988,0.004159,0.006017,0.916667
80,89 days,3.540508,1.881624,1.396771,0.007674,0.005613,0.007724,0.833333


Processing ABBV...


...

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,8 days,2.016052,1.419878,1.185519,0.005151,0.005538,0.005168,1.000000
1,11 days,1.464637,1.210222,0.921699,0.003995,0.002863,0.004007,1.000000
2,13 days,1.613959,1.270417,0.993683,0.004262,0.003666,0.004273,1.000000
3,14 days,1.962144,1.400765,1.066269,0.004489,0.003666,0.004482,1.000000
4,15 days,3.086290,1.756784,1.493876,0.006297,0.006340,0.006280,1.000000
5,18 days,2.798950,1.673006,1.394587,0.005860,0.005029,0.005840,1.000000
6,19 days,2.757639,1.660614,1.314548,0.005523,0.005029,0.005502,1.000000
7,20 days,2.763756,1.662455,1.325253,0.005564,0.005029,0.005543,1.000000
8,21 days,2.588043,1.608739,1.236128,0.005183,0.003889,0.005165,1.000000
9,22 days,1.867954,1.366731,1.064878,0.004447,0.003889,0.004435,1.000000


Processing XOM...


C:\Users\kahbe\AppData\Local\Temp\ipykernel_1320\2948041207.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prophet.rename(columns={'date': 'ds', 'close': 'y'}, inplace=True)


Optimal retraining period for XOM: 120 days
Performance metrics for XOM with optimal retraining period (120 days):


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,8 days,2.894528,1.701331,1.480058,0.014021,0.011581,0.014152,0.666667
1,11 days,2.896890,1.702025,1.482543,0.014008,0.011581,0.014139,0.666667
2,13 days,2.618530,1.618187,1.310209,0.012344,0.009025,0.012462,0.666667
3,14 days,2.997786,1.731412,1.462035,0.013840,0.013510,0.013930,0.666667
4,15 days,1.805998,1.343874,1.147757,0.010891,0.009850,0.010927,0.833333
5,18 days,1.196269,1.093741,0.981655,0.009374,0.009850,0.009384,1.000000
6,19 days,1.026212,1.013021,0.876016,0.008398,0.006923,0.008397,1.000000
7,20 days,0.988339,0.994153,0.808218,0.007768,0.006923,0.007765,1.000000
8,21 days,1.248584,1.117400,0.973604,0.009361,0.010756,0.009344,1.000000
9,22 days,1.855591,1.362201,1.115987,0.010825,0.010756,0.010779,0.833333


In [47]:
# Display the optimal retraining periods for each stock
print("Optimal retraining periods for each stock:")
for stock, period in optimal_retraining_periods.items():
    print(f"{stock}: {period}")

Optimal retraining periods for each stock:
AAPL: 60 days
ABBV: 120 days
ADBE: 90 days
AMZN: 60 days
AVGO: 120 days
BRK-B: 120 days
CRM: 60 days
COST: 60 days
CVX: 120 days
HD: 60 days
JNJ: 120 days
JPM: 30 days
LLY: 90 days
MA: 120 days
META: 60 days
MRK: 60 days
MSFT: 30 days
NVDA: 120 days
PG: 120 days
TSLA: 30 days
UNH: 120 days
V: 120 days
XOM: 120 days
